In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Corrección capas asociadas al temario Código 12: Avenida Torrencial</h4>
<p>Estos datos ya tienen un campo con nombre -Municipio-</p>

In [ ]:
ruta_capas_w_municipio = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"

arcpy.env.workspace = ruta_capas_w_municipio

dataset_intersect = 'Interseccion_Municipio'
columna_remover = 'Municipio_1'

for capas_w_mpio in arcpy.ListFeatureClasses(feature_dataset = dataset_intersect):
    if arcpy.ListFields(capas_w_mpio,columna_remover):
        arcpy.management.DeleteField(capas_w_mpio, 'Municipio')
        arcpy.management.AlterField(capas_w_mpio, 'Municipio_1', 'Municipio')
        print("Se borra y actualiza el campo -Municipio- en la capa: {0}".format(capas_w_mpio))        
    else:       
        print("La capa {0} no cuenta con una columna -Municipio_1-".format(capas_w_mpio))        

<h4>Creación de DataFrame para la unificación de la tabla -Relacion_Capa_Municipio-</h4>

In [ ]:
df_unificacion_tabla_capa_municipio = pd.DataFrame(columns=['codigo_capa','CODDANE','Municipio','codigo_tematica'])
print("---------------------------------------- Data frame creado ----------------------------------------")

<h4>Generación de DataFrame Global Total con la unificación de dos columnas principales</h4>
<ul>
  <li>codigo_capa</li>
  <li>coddane</li>
</ul>

In [ ]:
columna_analisis = 'Municipio'

for capas_w_mpio in arcpy.ListFeatureClasses(feature_dataset = dataset_intersect):
    if arcpy.ListFields(capas_w_mpio,columna_analisis):
        ruta_capas_w_mpio = os.path.join(ruta_capas_w_municipio, dataset_intersect, capas_w_mpio)
        df_capas_w_mpio = pd.DataFrame.spatial.from_featureclass(ruta_capas_w_mpio)
        capas_w_mpio_columnas_reducidas = df_capas_w_mpio.loc[:, ['codigo_capa','CODDANE','Municipio']]
        df_unificacion_tabla_capa_municipio = df_unificacion_tabla_capa_municipio.append(capas_w_mpio_columnas_reducidas)
        print("Se migra a DF: {0}".format(capas_w_mpio))
    else:
        print("La capa {0} no cuenta con la columna {1}, por lo cual no se adiciona al DF".format(capas_w_mpio,columna_analisis))

<h4>Lectura de la tabla previa -municipio-</h4>

In [ ]:
capa_municipio = r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\municipio'
df_municipio = pd.DataFrame.spatial.from_featureclass(capa_municipio)

<h4>Unificación Tabla -relación_capa_municipio-</h4>

In [ ]:
# Agrupamiento, lo que en SQL es Group by
df_tabla_capa = df_unificacion_tabla_capa_municipio.groupby(['codigo_capa','CODDANE','Municipio','codigo_tematica']).agg({'codigo_capa':'count', 'CODDANE':'count', 'Municipio':'count'})

# Se cambia el nombre de las columnas asociadas a los conteos (funcion -rename-)
df_tabla_capa.rename({'codigo_capa':'conteo_capa', 'CODDANE': 'conteo_CODDANE', 'Municipio':'conteo_municipio'}, axis = 1, inplace = True)

# Se resetan los índices
df_tabla_capa = df_tabla_capa.reset_index()

# Se cambia el nombre de la columna -CODDANE-
df_tabla_capa.rename({'CODDANE': 'codigo_municipio'}, axis = 1, inplace = True)

# Cambio de tipo de letra a -capitalize-
df_tabla_capa['Municipio'] = df_tabla_capa['Municipio'].str.capitalize()

# Se realiza la unión con la tabla -Municipio-
join_municipio = pd.merge(df_tabla_capa, df_municipio, how = 'inner', left_on = ['Municipio','codigo_municipio'], right_on = ['nombre_municipio','codigo_municipio_dane'])

# Estandarización de tabla
join_municipio = join_municipio[['codigo_municipio_y','codigo_capa']]

# Cambio de nombre al código de municipio
join_municipio.rename({'codigo_municipio_y': 'codigo_municipio'}, axis = 1, inplace = True)

# Generación del campo incremental de acuerdo al número de capas
RID = []

# Se suma uno por el header dentro del data frame
for id_rid in range(1, join_municipio.shape[0]+1):
    RID.append(id_rid)
 
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente   
join_municipio.loc[:,'RID'] = RID

# Se estandariza la tabla final
df_r_capa_municipio = join_municipio[['codigo_capa','codigo_municipio','RID']]
df_r_capa_municipio['codigo_capa'] = df_r_capa_municipio.loc[:,['codigo_capa']].astype('int')
df_r_capa_municipio['codigo_municipio'] = df_r_capa_municipio.loc[:,['codigo_municipio']].astype('int')

<h4>Exportado a .csv de la capa que relaciona -tbl_capa- con -municipio-</h4>
<p>Este .csv debe será migrado posteriormente cuando se cree la relación M:N<p>

In [ ]:
nombre_csv_capa_municipio = 'r_municipio_capa.csv'
df_r_capa_municipio.to_csv(nombre_csv_capa_municipio, index = False, encoding = 'UTF-8')